In [7]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

from predict_energy_behavior.models.joined_model import JoinedModel

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_parquet(
    "/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/processed/train/make_features/df_features.parquet",
    engine="fastparquet"
).dropna()
df["is_country_holiday"] = df["is_country_holiday"].astype(int)
df

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_historical,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target
row_id,,,,,,,,,,,,,,,,,,,,,
46970,0,0,1,0,2021-09-17 01:00:00,2021-09-17,260,1,17,5,...,1.447354,5.343272,0.594289,-7.673422,-0.730397,-0.895450,-8.414848,66.268753,0.000000,0.001
47092,0,0,1,0,2021-09-17 02:00:00,2021-09-17,260,2,17,5,...,2.014374,5.971542,-0.253379,-8.680022,0.718309,0.305286,-9.630847,62.298077,0.000000,0.000
47214,0,0,1,0,2021-09-17 03:00:00,2021-09-17,260,3,17,5,...,2.533695,2.236995,0.453719,-4.382467,-0.185719,-0.006285,-11.749646,-19.800087,0.000000,0.000
47336,0,0,1,0,2021-09-17 04:00:00,2021-09-17,260,4,17,5,...,2.836652,2.597171,0.397895,-4.545776,0.987074,0.005153,-11.411714,-19.661541,-0.003299,0.000
47458,0,0,1,0,2021-09-17 05:00:00,2021-09-17,260,5,17,5,...,3.025612,3.020513,0.554464,-4.207518,-2.007018,-0.507044,-12.682578,-21.136198,-0.028096,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017819,14,1,2,1,2023-05-31 19:00:00,2023-05-31,151,19,31,3,...,2.104196,3.820864,-0.298044,4.869689,0.113850,-1.154504,-2.619718,4.432365,11.855307,0.257
2017949,14,1,2,1,2023-05-31 20:00:00,2023-05-31,151,20,31,3,...,1.166394,2.733154,-0.665162,4.048730,0.498659,-1.076261,-3.987781,2.489141,16.528545,1.979
2018079,14,1,2,1,2023-05-31 21:00:00,2023-05-31,151,21,31,3,...,0.707611,1.804359,0.315522,3.727382,0.606830,-0.177199,-4.616146,3.756591,10.491406,6.911


In [38]:
df = pd.read_parquet(
    "/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/processed/train/make_features/df_features.parquet",
    engine="fastparquet"
).dropna()
df = df.loc[df["is_consumption"]==0] # select production
df

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_historical,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target
row_id,,,,,,,,,,,,,,,,,,,,,
46970,0,0,1,0,2021-09-17 01:00:00,2021-09-17,260,1,17,5,...,1.447354,5.343272,0.594289,-7.673422,-0.730397,-0.895450,-8.414848,66.268753,0.000000,0.001
47092,0,0,1,0,2021-09-17 02:00:00,2021-09-17,260,2,17,5,...,2.014374,5.971542,-0.253379,-8.680022,0.718309,0.305286,-9.630847,62.298077,0.000000,0.000
47214,0,0,1,0,2021-09-17 03:00:00,2021-09-17,260,3,17,5,...,2.533695,2.236995,0.453719,-4.382467,-0.185719,-0.006285,-11.749646,-19.800087,0.000000,0.000
47336,0,0,1,0,2021-09-17 04:00:00,2021-09-17,260,4,17,5,...,2.836652,2.597171,0.397895,-4.545776,0.987074,0.005153,-11.411714,-19.661541,-0.003299,0.000
47458,0,0,1,0,2021-09-17 05:00:00,2021-09-17,260,5,17,5,...,3.025612,3.020513,0.554464,-4.207518,-2.007018,-0.507044,-12.682578,-21.136198,-0.028096,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017818,14,1,2,0,2023-05-31 19:00:00,2023-05-31,151,19,31,3,...,2.104196,3.820864,-0.298044,4.869689,0.113850,-1.154504,-2.619718,4.432365,11.855307,35.026
2017948,14,1,2,0,2023-05-31 20:00:00,2023-05-31,151,20,31,3,...,1.166394,2.733154,-0.665162,4.048730,0.498659,-1.076261,-3.987781,2.489141,16.528545,14.867
2018078,14,1,2,0,2023-05-31 21:00:00,2023-05-31,151,21,31,3,...,0.707611,1.804359,0.315522,3.727382,0.606830,-0.177199,-4.616146,3.756591,10.491406,1.865


In [37]:
df["target"] / df["installed_capacity"]

row_id
46970      0.000001
47092      0.000000
47214      0.000000
47336      0.000000
47458      0.000012
             ...   
2017819    0.001045
2017949    0.008045
2018079    0.028093
2018209    0.042280
2018339    0.044142
Length: 1954342, dtype: float64

In [ ]:
train_features = [
    "temperature_historical",
    "dewpoint_historical",
    "rain_historical",
    "snowfall_historical",
    "cloudcover_total_historical",
    "cloudcover_low_historical",
    "cloudcover_mid_historical",
    "cloudcover_high_historical",
    "surface_solar_radiation_downwards_forecast",
    "windspeed_10m_historical",
    "10_metre_u_wind_component_historical",
    "10_metre_v_wind_component_historical",
    "cos(hour)",
    "sin(hour)",
    "cos(dayofyear)",
    "sin(dayofyear)",
    "fog_historical"
]

In [4]:
df_insp_p = df.loc[
    (df["county"]==0) &
    (df["product_type"]==1) &
    (df["is_business"]==1) &
    (df["is_consumption"]==0)
]
df_insp_p

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_historical,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target
row_id,,,,,,,,,,,,,,,,,,,,,
46978,0,1,1,0,2021-09-17 01:00:00,2021-09-17,260,1,17,5,...,1.447354,5.343272,0.594289,-7.673422,-0.730397,-0.895450,-8.414848,66.268753,0.000000,0.000
47100,0,1,1,0,2021-09-17 02:00:00,2021-09-17,260,2,17,5,...,2.014374,5.971542,-0.253379,-8.680022,0.718309,0.305286,-9.630847,62.298077,0.000000,0.000
47222,0,1,1,0,2021-09-17 03:00:00,2021-09-17,260,3,17,5,...,2.533695,2.236995,0.453719,-4.382467,-0.185719,-0.006285,-11.749646,-19.800087,0.000000,0.000
47344,0,1,1,0,2021-09-17 04:00:00,2021-09-17,260,4,17,5,...,2.836652,2.597171,0.397895,-4.545776,0.987074,0.005153,-11.411714,-19.661541,-0.003299,0.000
47466,0,1,1,0,2021-09-17 05:00:00,2021-09-17,260,5,17,5,...,3.025612,3.020513,0.554464,-4.207518,-2.007018,-0.507044,-12.682578,-21.136198,-0.028096,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017710,0,1,1,0,2023-05-31 19:00:00,2023-05-31,151,19,31,3,...,1.444504,3.078059,-1.183028,4.875686,0.008250,-1.406289,-1.086469,2.961163,32.079052,126.947
2017840,0,1,1,0,2023-05-31 20:00:00,2023-05-31,151,20,31,3,...,0.731187,2.048377,-0.801338,3.132859,0.011203,-1.185897,-5.386113,0.446676,23.237328,27.563
2017970,0,1,1,0,2023-05-31 21:00:00,2023-05-31,151,21,31,3,...,0.131821,1.522847,0.099047,3.733331,0.665836,-0.819711,-5.548689,1.733130,12.428007,5.091


In [29]:
df_insp_p.loc[df_insp_p.isna().any(axis=1)]

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_historical,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target
row_id,,,,,,,,,,,,,,,,,,,,,


In [34]:
def viz_weather_and_predictions_for_month(df, year: int, month: int):
    t_start = pd.Timestamp(year=year, month=month, day=1)
    t_end = t_start + pd.DateOffset(months=1)
    df_this_year = df.loc[
        (df["datetime"] > t_start) &
        (df["datetime"] < t_end)
    ]
    df_last_year = df.loc[
        (df["datetime"] > (t_start - pd.DateOffset(years=1))) &
        (df["datetime"] < (t_end - pd.DateOffset(years=1)))
    ]
    df_last_year = df_last_year.assign(**{"datetime": df_last_year["datetime"] + pd.DateOffset(years=1)})
    
    fig = make_subplots(cols=1, rows=5, subplot_titles=["Temperature", "Rain", "Snowfall", "I"], shared_xaxes=True)
    
    for df, opacity in zip([df_this_year, df_last_year], [1, 0.5]):
        # Temperature
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["temperature_forecast"], marker_color="blue", name="forecast", legendgroup=1, opacity=opacity),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["temperature_historical"], marker_color="red", name="historical", legendgroup=2, opacity=opacity),
            row=1, col=1
        )

        # Rain
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["rain_forecast"], marker_color="blue", name="forecast", legendgroup=1, showlegend=False, opacity=opacity),
            row=2, col=1
        )
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["rain_historical"], marker_color="red", name="historical", legendgroup=2, showlegend=False, opacity=opacity),
            row=2, col=1
        )

        # Snow
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["snowfall_forecast"], marker_color="blue", name="forecast", legendgroup=1, showlegend=False, opacity=opacity),
            row=3, col=1
        )
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["snowfall_historical"], marker_color="red", name="historical", legendgroup=2, showlegend=False, opacity=opacity),
            row=3, col=1
        )

        # I
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["surface_solar_radiation_downwards_forecast"], marker_color="blue", name="forecast", legendgroup=1, showlegend=False, opacity=opacity),
            row=4, col=1
        )


        # target
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["target"] / df["installed_capacity"], marker_color="green", name="target", showlegend=True, opacity=opacity),
            row=5, col=1
        )
        fig.add_trace(
            go.Scatter(x=df["datetime"], y=df["predictions"] / df["installed_capacity"], marker_color="orange", name="predictions", showlegend=True, opacity=opacity),
            row=5, col=1
        )

    fig.update_layout(height=800)

    return fig

In [8]:
model = JoinedModel.load(Path("/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/outputs/train/lgbms-ens+diff-lgbms/model_joined-f-2022-12"))
model

In [10]:

insp_p_preds = model.predict(df_insp_p)
insp_p_preds

/beegfs/.global0/ws/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/src/predict_energy_behavior/models/joined_model.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return pd.concat([preds_p, preds_c]).loc[X.index].to_numpy()


array([3.92594136e-03, 5.37268855e-03, 4.62824955e-03, ...,
       1.44010311e+01, 4.37374990e-01, 3.70876741e-02])

In [11]:
df_insp_p["predictions"] = insp_p_preds

/tmp/ipykernel_240236/3417454827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_insp_p["predictions"] = insp_p_preds


In [15]:
df_insp_p

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target,predictions
row_id,,,,,,,,,,,,,,,,,,,,,
46978,0,1,1,0,2021-09-17 01:00:00,2021-09-17,260,1,17,5,...,5.343272,0.594289,-7.673422,-0.730397,-0.895450,-8.414848,66.268753,0.000000,0.000,0.003926
47100,0,1,1,0,2021-09-17 02:00:00,2021-09-17,260,2,17,5,...,5.971542,-0.253379,-8.680022,0.718309,0.305286,-9.630847,62.298077,0.000000,0.000,0.005373
47222,0,1,1,0,2021-09-17 03:00:00,2021-09-17,260,3,17,5,...,2.236995,0.453719,-4.382467,-0.185719,-0.006285,-11.749646,-19.800087,0.000000,0.000,0.004628
47344,0,1,1,0,2021-09-17 04:00:00,2021-09-17,260,4,17,5,...,2.597171,0.397895,-4.545776,0.987074,0.005153,-11.411714,-19.661541,-0.003299,0.000,-0.110333
47466,0,1,1,0,2021-09-17 05:00:00,2021-09-17,260,5,17,5,...,3.020513,0.554464,-4.207518,-2.007018,-0.507044,-12.682578,-21.136198,-0.028096,0.000,-1.359544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017710,0,1,1,0,2023-05-31 19:00:00,2023-05-31,151,19,31,3,...,3.078059,-1.183028,4.875686,0.008250,-1.406289,-1.086469,2.961163,32.079052,126.947,408.345502
2017840,0,1,1,0,2023-05-31 20:00:00,2023-05-31,151,20,31,3,...,2.048377,-0.801338,3.132859,0.011203,-1.185897,-5.386113,0.446676,23.237328,27.563,69.221172
2017970,0,1,1,0,2023-05-31 21:00:00,2023-05-31,151,21,31,3,...,1.522847,0.099047,3.733331,0.665836,-0.819711,-5.548689,1.733130,12.428007,5.091,14.401031


In [35]:
viz_weather_and_predictions_for_month(df_insp_p, 2023, 4)